# Unit 2 Integrating Machine Learning Models with FastAPI for Predictions

# Welcome to "Model Serving with FastAPI" - Lesson 2

Welcome back to our "Model Serving with FastAPI" course\! In this second lesson, we're advancing our journey to build a robust diamond price prediction API. In our previous lesson, we established the foundation by creating a basic FastAPI application with a root endpoint and a health check. Today, we'll take a significant step forward by integrating our machine learning model into the API and creating an endpoint for making predictions.

By the end of this lesson, you'll have a functional prediction endpoint that validates input data, processes it through your machine learning model, and returns diamond price predictions to users. This represents the core functionality of our model serving application. Let's begin by understanding how to properly handle and validate input data for our machine learning model\!

-----

## Understanding Pydantic Models for Data Validation

Before diving into code, let's explore an essential component for building robust APIs: **data validation**. When serving machine learning models, ensuring that input data meets your expectations is crucial for preventing errors, providing clear feedback, and maintaining data integrity.

FastAPI leverages **Pydantic** for data validation, which validates data using Python type annotations. Here's how we can define the expected structure of diamond feature inputs:

```python
from pydantic import BaseModel, Field

class DiamondFeatures(BaseModel):
    """Pydantic model for diamond features input"""
    carat: float = Field(..., gt=0, description="Weight of the diamond (0.2-5.0)")
    cut: str = Field(..., description="Quality of the cut (Fair, Good, Very Good, Premium, Ideal)")
    color: str = Field(..., description="Diamond color, from J (worst) to D (best)")
    clarity: str = Field(..., description="Clarity (I1, SI2, SI1, VS2, VS1, VVS2, VVS1, IF)")
    depth: float = Field(..., gt=0, description="Total depth percentage")
    table: float = Field(..., gt=0, description="Width of top relative to widest point")
    x: float = Field(..., gt=0, description="Length in mm")
    y: float = Field(..., gt=0, description="Width in mm")
    z: float = Field(..., gt=0, description="Depth in mm")
```

In this code, we're creating a `DiamondFeatures` class that inherits from Pydantic's `BaseModel`. Each attribute has a type annotation and additional validation rules. For example, `carat: float = Field(..., gt=0)` specifies that carat must be a positive floating-point number, and the ellipsis (`...`) indicates the field is required. The `description` parameter documents what each field represents in the auto-generated API docs.

We'll also create a model for our prediction response:

```python
class PredictionResponse(BaseModel):
    """Pydantic model for prediction response"""
    predicted_price: float
    diamond_features: DiamondFeatures
```

This response model ensures a consistent output format that includes both the prediction and the original input features for reference. When you use these Pydantic models, FastAPI automatically validates incoming requests, converts data to the correct types, and generates helpful error messages when validation fails.

-----

## Loading and Managing ML Models

Now that you have your data validation set up, let's tackle one of the key challenges of model serving: **efficiently loading and managing machine learning models**. Since models can be memory-intensive, you need a strategy to handle them effectively in your API.

Let's implement a dependency function that loads our model and preprocessor:

```python
def get_model():
    """Dependency function to load and return the model and preprocessor on demand"""
    try:
        # Ensure model directory exists
        os.makedirs(MODEL_DIR, exist_ok=True)
                
        # Construct file paths
        model_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}.joblib")
        preprocessor_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}_preprocessor.joblib")
                
        # Check if model exists
        if not os.path.exists(model_path) or not os.path.exists(preprocessor_path):
            raise HTTPException(
                status_code=404, 
                detail="Model files not found. Please train and save a model first."
            )
            
        # Load model and preprocessor
        model, preprocessor, _ = load_model_with_metadata(MODEL_DIR, MODEL_NAME)
        return model, preprocessor
    except Exception as e:
        raise HTTPException(status_code=503, detail=f"Error loading model: {str(e)}")
```

This function performs several critical tasks: it verifies the model files exist, loads them using the `load_model_with_metadata` helper function (that we implemented in the previous course), and handles any errors by converting them to appropriate HTTP responses. When your API receives a prediction request, this function will provide the necessary model resources to process it.

-----

## Implementing FastAPI Dependency Injection

With our model loading functions in place, let's look at how to efficiently make these models available to your API endpoints using FastAPI's powerful **dependency injection** system. Dependency injection allows you to:

  * Share resources between endpoints
  * Manage expensive resources efficiently
  * Simplify endpoint function signatures
  * Facilitate testing through dependency overrides

Here's how you'll use it for your prediction endpoint:

```python
from fastapi import FastAPI, HTTPException, Depends

@app.post("/predict", response_model=PredictionResponse)
async def predict_price(
    diamond: DiamondFeatures, 
    model_data: tuple = Depends(get_model)
):
    """Predict the price of a diamond based on its features"""
    # The model and preprocessor are injected through the dependency
    model, preprocessor = model_data

    # We'll implement the prediction logic next
```

Notice the `model_data: tuple = Depends(get_model)` parameter in our endpoint function. This tells FastAPI to call our `get_model()` function before executing the endpoint and pass the result to the function. FastAPI even caches the dependency result during the request lifecycle, preventing redundant model loading if multiple endpoints use the same model.

This approach creates a clean separation of concerns: your model loading logic stays independent from your prediction logic, making your code more maintainable and testable.

-----

## Creating the Prediction Endpoint

Let's complete the implementation of the **prediction endpoint**:

```python
@app.post("/predict", response_model=PredictionResponse)
async def predict_price(
    diamond: DiamondFeatures, 
    model_data: tuple = Depends(get_model)
):
    """Predict the price of a diamond based on its features"""
    # Extract model and preprocessor
    model, preprocessor = model_data

    # Convert Pydantic model to DataFrame for preprocessing
    input_df = pd.DataFrame([diamond.dict()])

    try:
        # Preprocess the input features
        X_processed = preprocessor.transform(input_df)

        # Generate prediction
        prediction = model.predict(X_processed)[0]

        # Return formatted response
        return {
            "predicted_price": float(prediction),
            "diamond_features": diamond
        }
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Prediction error: {str(e)}"
        )
```

This endpoint function:

  * Receives validated diamond features through our Pydantic model.
  * Gets the model and preprocessor through dependency injection.
  * Converts the input to a `pandas` DataFrame (the format expected by most scikit-learn preprocessors).
  * Applies the same preprocessing steps used during training.
  * Makes a prediction using the model.
  * Returns the predicted price along with the original features.

The `response_model=PredictionResponse` parameter ensures that FastAPI will validate and format our response according to the defined schema, maintaining consistency in your API responses.

-----

## Error Handling for Model Serving

A robust API needs comprehensive **error handling** to gracefully manage the various issues that can arise during model serving. Let's examine the error handling strategies implemented in your diamond price prediction API:

```python
# In get_model dependency:
if not os.path.exists(model_path):
    raise HTTPException(
        status_code=404, 
        detail="Model files not found. Please train and save a model first."
    )

# In prediction endpoint:
try:
    # Prediction logic...
except Exception as e:
    raise HTTPException(
        status_code=500,
        detail=f"Prediction error: {str(e)}"
    )
```

This multi-layered approach addresses three key types of errors:

  * **Input validation errors:** FastAPI and Pydantic automatically handle these, returning detailed 422 Unprocessable Entity responses when diamond features don't meet your specifications.
  * **Model availability errors:** Your dependency function checks if model files exist and returns a clear 404 Not Found response when they don't, helping API users understand they need to train a model first.
  * **Processing errors:** Your prediction endpoint catches any exceptions during preprocessing or prediction, returning a 500 Internal Server Error with details about what went wrong.

-----

## Conclusion and Next Steps

Congratulations\! You've successfully transformed a basic FastAPI application into a functional machine learning service capable of predicting diamond prices. You've implemented data validation with Pydantic, created efficient model loading strategies through dependency injection, built a prediction endpoint that processes inputs and returns results, and established robust error handling to make your API reliable and user-friendly.

Now it's time to test what you learned with some hands-on practice. Happy coding\!

## Converting Features for Prediction

Welcome to your first hands-on practice in integrating machine learning models with FastAPI! In the previous lesson, you learned about creating a prediction endpoint and handling data validation using Pydantic models. Now, it's time to put that knowledge into action.

Your first goal here is to locate the placeholder in the predict_price endpoint and insert the correct code to transform the diamond object into a pandas DataFrame.

```python
import os
import pandas as pd
import uvicorn
from fastapi import FastAPI, HTTPException, Depends
from fastapi.responses import RedirectResponse
from pydantic import BaseModel, Field

from model import load_model_with_metadata

app = FastAPI(
    title="Diamond Price Prediction API",
    description="API for predicting diamond prices based on diamond characteristics",
    version="1.0.0"
)

# Model configuration
MODEL_DIR = "saved_models"
MODEL_NAME = "model"

class DiamondFeatures(BaseModel):
    """
    Pydantic model for diamond features input
    """
    carat: float = Field(..., gt=0, description="Weight of the diamond (0.2-5.0)")
    cut: str = Field(..., description="Quality of the cut (Fair, Good, Very Good, Premium, Ideal)")
    color: str = Field(..., description="Diamond color, from J (worst) to D (best)")
    clarity: str = Field(..., description="Clarity of the diamond (I1, SI2, SI1, VS2, VS1, VVS2, VVS1, IF)")
    depth: float = Field(..., gt=0, description="Total depth percentage")
    table: float = Field(..., gt=0, description="Width of top of diamond relative to widest point")
    x: float = Field(..., gt=0, description="Length in mm")
    y: float = Field(..., gt=0, description="Width in mm")
    z: float = Field(..., gt=0, description="Depth in mm")

class PredictionResponse(BaseModel):
    """
    Pydantic model for prediction response
    """
    predicted_price: float
    diamond_features: DiamondFeatures

def get_model():
    """
    Dependency function to load and return the model and preprocessor on demand
    """
    try:
        os.makedirs(MODEL_DIR, exist_ok=True)
        model_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}.joblib")
        preprocessor_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}_preprocessor.joblib")
        
        # Check if model files exist
        if not os.path.exists(model_path) or not os.path.exists(preprocessor_path):
            raise HTTPException(
                status_code=404, 
                detail="Model files not found. Please train and save a model first."
            )
            
        model, preprocessor, _ = load_model_with_metadata(MODEL_DIR, MODEL_NAME)
        return model, preprocessor
    except Exception as e:
        raise HTTPException(status_code=503, detail=f"Error loading model: {str(e)}")

@app.get("/")
async def root():
    """
    Root endpoint, redirects to API documentation
    """
    return RedirectResponse(url="/docs")

@app.get("/health")
async def health_check():
    """
    Health check endpoint to verify the API is running correctly
    """
    model_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}.joblib")
    preprocessor_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}_preprocessor.joblib")
    model_status = "available" if (os.path.exists(model_path) and os.path.exists(preprocessor_path)) else "not available"
    
    return {
        "status": "healthy",
        "api_version": "1.0.0",
        "model_status": model_status
    }

@app.post("/predict", response_model=PredictionResponse)
async def predict_price(diamond: DiamondFeatures, model_data: tuple = Depends(get_model)):
    """
    Predict the price of a diamond based on its features
    """
    # Use the dependency-injected model and preprocessor
    model, preprocessor = model_data
    
    # TODO: Convert the diamond Pydantic model to a pandas DataFrame
    input_df = _____________    
    
    try:
        # Preprocess the input using the same preprocessor from training
        X_processed = preprocessor.transform(input_df)
        
        # Make prediction
        prediction = model.predict(X_processed)[0]
        
        # Return the prediction along with the provided features
        return {
            "predicted_price": float(prediction),
            "diamond_features": diamond
        }
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Prediction error: {str(e)}"
        )

if __name__ == "__main__":
    uvicorn.run("main:app", host="0.0.0.0", port=3000, reload=True)

```

```python
import os
import pandas as pd
import uvicorn
from fastapi import FastAPI, HTTPException, Depends
from fastapi.responses import RedirectResponse
from pydantic import BaseModel, Field

from model import load_model_with_metadata

app = FastAPI(
    title="Diamond Price Prediction API",
    description="API for predicting diamond prices based on diamond characteristics",
    version="1.0.0"
)

# Model configuration
MODEL_DIR = "saved_models"
MODEL_NAME = "model"

class DiamondFeatures(BaseModel):
    """
    Pydantic model for diamond features input
    """
    carat: float = Field(..., gt=0, description="Weight of the diamond (0.2-5.0)")
    cut: str = Field(..., description="Quality of the cut (Fair, Good, Very Good, Premium, Ideal)")
    color: str = Field(..., description="Diamond color, from J (worst) to D (best)")
    clarity: str = Field(..., description="Clarity of the diamond (I1, SI2, SI1, VS2, VS1, VVS2, VVS1, IF)")
    depth: float = Field(..., gt=0, description="Total depth percentage")
    table: float = Field(..., gt=0, description="Width of top of diamond relative to widest point")
    x: float = Field(..., gt=0, description="Length in mm")
    y: float = Field(..., gt=0, description="Width in mm")
    z: float = Field(..., gt=0, description="Depth in mm")

class PredictionResponse(BaseModel):
    """
    Pydantic model for prediction response
    """
    predicted_price: float
    diamond_features: DiamondFeatures

def get_model():
    """
    Dependency function to load and return the model and preprocessor on demand
    """
    try:
        os.makedirs(MODEL_DIR, exist_ok=True)
        model_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}.joblib")
        preprocessor_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}_preprocessor.joblib")
        
        # Check if model files exist
        if not os.path.exists(model_path) or not os.path.exists(preprocessor_path):
            raise HTTPException(
                status_code=404, 
                detail="Model files not found. Please train and save a model first."
            )
            
        model, preprocessor, _ = load_model_with_metadata(MODEL_DIR, MODEL_NAME)
        return model, preprocessor
    except Exception as e:
        raise HTTPException(status_code=503, detail=f"Error loading model: {str(e)}")

@app.get("/")
async def root():
    """
    Root endpoint, redirects to API documentation
    """
    return RedirectResponse(url="/docs")

@app.get("/health")
async def health_check():
    """
    Health check endpoint to verify the API is running correctly
    """
    model_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}.joblib")
    preprocessor_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}_preprocessor.joblib")
    model_status = "available" if (os.path.exists(model_path) and os.path.exists(preprocessor_path)) else "not available"
    
    return {
        "status": "healthy",
        "api_version": "1.0.0",
        "model_status": model_status
    }

@app.post("/predict", response_model=PredictionResponse)
async def predict_price(diamond: DiamondFeatures, model_data: tuple = Depends(get_model)):
    """
    Predict the price of a diamond based on its features
    """
    # Use the dependency-injected model and preprocessor
    model, preprocessor = model_data
    
    # TODO: Convert the diamond Pydantic model to a pandas DataFrame
    input_df = pd.DataFrame([diamond.dict()])
    
    try:
        # Preprocess the input using the same preprocessor from training
        X_processed = preprocessor.transform(input_df)
        
        # Make prediction
        prediction = model.predict(X_processed)[0]
        
        # Return the prediction along with the provided features
        return {
            "predicted_price": float(prediction),
            "diamond_features": diamond
        }
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Prediction error: {str(e)}"
        )

if __name__ == "__main__":
    uvicorn.run("main:app", host="0.0.0.0", port=3000, reload=True)
```

## Loading and Managing ML Models

Welcome back! In the previous exercise, you successfully transformed diamond features into a pandas DataFrame. Now, let's ensure your model is ready to make predictions by completing the get_model function. This function is essential as it loads the machine learning model and its preprocessor, making them available for your API to use.

Your objective is to fill in the missing parts of the get_model function. Here's what you need to do:

Create the model directory if it doesn't exist.
Construct the file paths for the model and preprocessor.
Check if the model and preprocessor files exist. If they don't, raise an HTTP exception to inform users that the model needs to be trained and saved first.
Load the model and preprocessor using the load_model_with_metadata function.
Remember, this function is a dependency for your prediction endpoint, so it needs to be robust and handle errors gracefully. Dive in and ensure your API is ready to serve accurate predictions!

```python
from fastapi import FastAPI, HTTPException, Depends
from fastapi.responses import RedirectResponse
from pydantic import BaseModel, Field
import uvicorn
import os
import pandas as pd

from model import load_model_with_metadata

# Initialize FastAPI app
app = FastAPI(
    title="Diamond Price Prediction API",
    description="API for predicting diamond prices based on diamond characteristics",
    version="1.0.0"
)

# Model configuration
MODEL_DIR = "saved_models"
MODEL_NAME = "model"

class DiamondFeatures(BaseModel):
    """
    Pydantic model for diamond features input
    """
    carat: float = Field(..., gt=0, description="Weight of the diamond (0.2-5.0)")
    cut: str = Field(..., description="Quality of the cut (Fair, Good, Very Good, Premium, Ideal)")
    color: str = Field(..., description="Diamond color, from J (worst) to D (best)")
    clarity: str = Field(..., description="Clarity of the diamond (I1, SI2, SI1, VS2, VS1, VVS2, VVS1, IF)")
    depth: float = Field(..., gt=0, description="Total depth percentage")
    table: float = Field(..., gt=0, description="Width of top of diamond relative to widest point")
    x: float = Field(..., gt=0, description="Length in mm")
    y: float = Field(..., gt=0, description="Width in mm")
    z: float = Field(..., gt=0, description="Depth in mm")

class PredictionResponse(BaseModel):
    """
    Pydantic model for prediction response
    """
    predicted_price: float
    diamond_features: DiamondFeatures

def get_model():
    """
    Dependency function to load and return the model and preprocessor on demand
    """
    try:
        # TODO: Create the model directory if it doesn't exist
        ____________
        
        # TODO: Construct the file paths for the model and preprocessor
        model_path = ____________
        preprocessor_path = ____________
        
        # TODO: Check if model files exist and raise an appropriate exception if they don't
        if ____________:
            raise HTTPException(
                status_code=404, 
                detail="Model files not found. Please train and save a model first."
            )
            
        # TODO: Load the model and preprocessor using the load_model_with_metadata function
        model, preprocessor, _ = ____________
        
        return model, preprocessor
    except Exception as e:
        raise HTTPException(status_code=503, detail=f"Error loading model: {str(e)}")

@app.get("/")
async def root():
    """
    Root endpoint, redirects to API documentation
    """
    return RedirectResponse(url="/docs")

@app.get("/health")
async def health_check():
    """
    Health check endpoint to verify the API is running correctly
    """
    model_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}.joblib")
    preprocessor_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}_preprocessor.joblib")
    model_status = "available" if (os.path.exists(model_path) and os.path.exists(preprocessor_path)) else "not available"
    
    return {
        "status": "healthy",
        "api_version": "1.0.0",
        "model_status": model_status
    }

@app.post("/predict", response_model=PredictionResponse)
async def predict_price(diamond: DiamondFeatures, model_data: tuple = Depends(get_model)):
    """
    Predict the price of a diamond based on its features
    """
    # Use the dependency-injected model and preprocessor
    model, preprocessor = model_data
    
    # Convert input to DataFrame for preprocessing
    input_df = pd.DataFrame([diamond.dict()])
    
    try:
        # Preprocess the input using the same preprocessor from training
        X_processed = preprocessor.transform(input_df)
        
        # Make prediction
        prediction = model.predict(X_processed)[0]
        
        # Return the prediction
        return {
            "predicted_price": float(prediction),
            "diamond_features": diamond
        }
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Prediction error: {str(e)}"
        )


if __name__ == "__main__":
    # Run the application with uvicorn when script is executed directly
    uvicorn.run("main:app", host="0.0.0.0", port=3000, reload=True) 

```

```python
from fastapi import FastAPI, HTTPException, Depends
from fastapi.responses import RedirectResponse
from pydantic import BaseModel, Field
import uvicorn
import os
import pandas as pd

from model import load_model_with_metadata

# Initialize FastAPI app
app = FastAPI(
    title="Diamond Price Prediction API",
    description="API for predicting diamond prices based on diamond characteristics",
    version="1.0.0"
)

# Model configuration
MODEL_DIR = "saved_models"
MODEL_NAME = "model"

class DiamondFeatures(BaseModel):
    """
    Pydantic model for diamond features input
    """
    carat: float = Field(..., gt=0, description="Weight of the diamond (0.2-5.0)")
    cut: str = Field(..., description="Quality of the cut (Fair, Good, Very Good, Premium, Ideal)")
    color: str = Field(..., description="Diamond color, from J (worst) to D (best)")
    clarity: str = Field(..., description="Clarity of the diamond (I1, SI2, SI1, VS2, VS1, VVS2, VVS1, IF)")
    depth: float = Field(..., gt=0, description="Total depth percentage")
    table: float = Field(..., gt=0, description="Width of top of diamond relative to widest point")
    x: float = Field(..., gt=0, description="Length in mm")
    y: float = Field(..., gt=0, description="Width in mm")
    z: float = Field(..., gt=0, description="Depth in mm")

class PredictionResponse(BaseModel):
    """
    Pydantic model for prediction response
    """
    predicted_price: float
    diamond_features: DiamondFeatures

def get_model():
    """
    Dependency function to load and return the model and preprocessor on demand
    """
    try:
        # TODO: Create the model directory if it doesn't exist
        os.makedirs(MODEL_DIR, exist_ok=True)
        
        # TODO: Construct the file paths for the model and preprocessor
        model_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}.joblib")
        preprocessor_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}_preprocessor.joblib")
        
        # TODO: Check if model files exist and raise an appropriate exception if they don't
        if not os.path.exists(model_path) or not os.path.exists(preprocessor_path):
            raise HTTPException(
                status_code=404, 
                detail="Model files not found. Please train and save a model first."
            )
            
        # TODO: Load the model and preprocessor using the load_model_with_metadata function
        model, preprocessor, _ = load_model_with_metadata(MODEL_DIR, MODEL_NAME)
        
        return model, preprocessor
    except Exception as e:
        raise HTTPException(status_code=503, detail=f"Error loading model: {str(e)}")

@app.get("/")
async def root():
    """
    Root endpoint, redirects to API documentation
    """
    return RedirectResponse(url="/docs")

@app.get("/health")
async def health_check():
    """
    Health check endpoint to verify the API is running correctly
    """
    model_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}.joblib")
    preprocessor_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}_preprocessor.joblib")
    model_status = "available" if (os.path.exists(model_path) and os.path.exists(preprocessor_path)) else "not available"
    
    return {
        "status": "healthy",
        "api_version": "1.0.0",
        "model_status": model_status
    }

@app.post("/predict", response_model=PredictionResponse)
async def predict_price(diamond: DiamondFeatures, model_data: tuple = Depends(get_model)):
    """
    Predict the price of a diamond based on its features
    """
    # Use the dependency-injected model and preprocessor
    model, preprocessor = model_data
    
    # Convert input to DataFrame for preprocessing
    input_df = pd.DataFrame([diamond.dict()])
    
    try:
        # Preprocess the input using the same preprocessor from training
        X_processed = preprocessor.transform(input_df)
        
        # Make prediction
        prediction = model.predict(X_processed)[0]
        
        # Return the prediction
        return {
            "predicted_price": float(prediction),
            "diamond_features": diamond
        }
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Prediction error: {str(e)}"
        )


if __name__ == "__main__":
    # Run the application with uvicorn when script is executed directly
    uvicorn.run("main:app", host="0.0.0.0", port=3000, reload=True)
```

## Defining Diamond Data Structure

Welcome back!

Your goal in this exercise is to implement the DiamondFeatures class, which will serve as the blueprint for the diamond data your API will receive. This class should inherit from pydantic's BaseModel and include fields for each diamond attribute: carat, cut, color, clarity, depth, table, x, y, and z.

Each field should have a type annotation and validation rules using Field, ensuring the data is both accurate and meaningful. For instance, the carat field should be a positive float, and the cut should be a string describing the quality.

By completing this, you'll solidify your understanding of data validation and ensure your API can handle incoming requests with confidence. Dive in and craft a robust DiamondFeatures model!


```python
"""
FastAPI Application with Model Integration for Diamond Price Prediction

This module extends the basic FastAPI application to include endpoints for
loading the ML model and making diamond price predictions.
"""

from fastapi import FastAPI, HTTPException, Depends
from fastapi.responses import RedirectResponse
from pydantic import BaseModel, Field
import uvicorn
import os
import pandas as pd

from model import load_model_with_metadata

# Initialize FastAPI app
app = FastAPI(
    title="Diamond Price Prediction API",
    description="API for predicting diamond prices based on diamond characteristics",
    version="1.0.0"
)

# Model configuration
MODEL_DIR = "saved_models"
MODEL_NAME = "model"

# TODO: Implement the DiamondFeatures class that inherits from BaseModel
# This class should define all the diamond attributes with appropriate type annotations
# and validation rules using Field


class PredictionResponse(BaseModel):
    """
    Pydantic model for prediction response
    """
    predicted_price: float
    diamond_features: DiamondFeatures  # This will use your DiamondFeatures class

def get_model():
    """
    Dependency function to load and return the model and preprocessor on demand
    """
    try:
        os.makedirs(MODEL_DIR, exist_ok=True)
        model_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}.joblib")
        preprocessor_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}_preprocessor.joblib")
        
        # Check if model exists
        if not os.path.exists(model_path) or not os.path.exists(preprocessor_path):
            raise HTTPException(
                status_code=404, 
                detail="Model files not found. Please train and save a model first."
            )
            
        model, preprocessor, _ = load_model_with_metadata(MODEL_DIR, MODEL_NAME)
        return model, preprocessor
    except Exception as e:
        raise HTTPException(status_code=503, detail=f"Error loading model: {str(e)}")

@app.get("/")
async def root():
    """
    Root endpoint, redirects to API documentation
    """
    return RedirectResponse(url="/docs")

@app.get("/health")
async def health_check():
    """
    Health check endpoint to verify the API is running correctly
    """
    model_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}.joblib")
    preprocessor_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}_preprocessor.joblib")
    model_status = "available" if (os.path.exists(model_path) and os.path.exists(preprocessor_path)) else "not available"
    
    return {
        "status": "healthy",
        "api_version": "1.0.0",
        "model_status": model_status
    }

@app.post("/predict", response_model=PredictionResponse)
async def predict_price(diamond: DiamondFeatures, model_data: tuple = Depends(get_model)):
    """
    Predict the price of a diamond based on its features
    """
    # Use the dependency-injected model and preprocessor
    model, preprocessor = model_data
    
    # Convert input to DataFrame for preprocessing
    input_df = pd.DataFrame([diamond.dict()])
    
    try:
        # Preprocess the input using the same preprocessor from training
        X_processed = preprocessor.transform(input_df)
        
        # Make prediction
        prediction = model.predict(X_processed)[0]
        
        # Return the prediction
        return {
            "predicted_price": float(prediction),
            "diamond_features": diamond
        }
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Prediction error: {str(e)}"
        )


if __name__ == "__main__":
    # Run the application with uvicorn when script is executed directly
    uvicorn.run("main:app", host="0.0.0.0", port=3000, reload=True) 

```

```python
"""
FastAPI Application with Model Integration for Diamond Price Prediction

This module extends the basic FastAPI application to include endpoints for
loading the ML model and making diamond price predictions.
"""

from fastapi import FastAPI, HTTPException, Depends
from fastapi.responses import RedirectResponse
from pydantic import BaseModel, Field, PositiveFloat, constr
import uvicorn
import os
import pandas as pd

from model import load_model_with_metadata

# Initialize FastAPI app
app = FastAPI(
    title="Diamond Price Prediction API",
    description="API for predicting diamond prices based on diamond characteristics",
    version="1.0.0"
)

# Model configuration
MODEL_DIR = "saved_models"
MODEL_NAME = "model"

# TODO: Implement the DiamondFeatures class that inherits from BaseModel
# This class should define all the diamond attributes with appropriate type annotations
# and validation rules using Field


class DiamondFeatures(BaseModel):
    """
    Pydantic model for validating diamond features input
    """
    carat: PositiveFloat = Field(..., description="The weight of the diamond in carats.", gt=0)
    cut: str = Field(..., description="The quality of the diamond's cut.", pattern=r'^(Fair|Good|Very Good|Premium|Ideal)$')
    color: str = Field(..., description="The color of the diamond.", pattern=r'^[J-D]$')
    clarity: str = Field(..., description="The clarity of the diamond.", pattern=r'^(I1|SI2|SI1|VS2|VS1|VVS2|VVS1|IF)$')
    depth: float = Field(..., description="The total depth percentage.", gt=0)
    table: float = Field(..., description="The width of the top facet relative to the widest point.", gt=0)
    x: float = Field(..., description="The length of the diamond in mm.", gt=0)
    y: float = Field(..., description="The width of the diamond in mm.", gt=0)
    z: float = Field(..., description="The depth of the diamond in mm.", gt=0)


class PredictionResponse(BaseModel):
    """
    Pydantic model for prediction response
    """
    predicted_price: float
    diamond_features: DiamondFeatures  # This will use your DiamondFeatures class

def get_model():
    """
    Dependency function to load and return the model and preprocessor on demand
    """
    try:
        os.makedirs(MODEL_DIR, exist_ok=True)
        model_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}.joblib")
        preprocessor_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}_preprocessor.joblib")
        
        # Check if model exists
        if not os.path.exists(model_path) or not os.path.exists(preprocessor_path):
            raise HTTPException(
                status_code=404, 
                detail="Model files not found. Please train and save a model first."
            )
            
        model, preprocessor, _ = load_model_with_metadata(MODEL_DIR, MODEL_NAME)
        return model, preprocessor
    except Exception as e:
        raise HTTPException(status_code=503, detail=f"Error loading model: {str(e)}")

@app.get("/")
async def root():
    """
    Root endpoint, redirects to API documentation
    """
    return RedirectResponse(url="/docs")

@app.get("/health")
async def health_check():
    """
    Health check endpoint to verify the API is running correctly
    """
    model_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}.joblib")
    preprocessor_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}_preprocessor.joblib")
    model_status = "available" if (os.path.exists(model_path) and os.path.exists(preprocessor_path)) else "not available"
    
    return {
        "status": "healthy",
        "api_version": "1.0.0",
        "model_status": model_status
    }

@app.post("/predict", response_model=PredictionResponse)
async def predict_price(diamond: DiamondFeatures, model_data: tuple = Depends(get_model)):
    """
    Predict the price of a diamond based on its features
    """
    # Use the dependency-injected model and preprocessor
    model, preprocessor = model_data
    
    # Convert input to DataFrame for preprocessing
    input_df = pd.DataFrame([diamond.dict()])
    
    try:
        # Preprocess the input using the same preprocessor from training
        X_processed = preprocessor.transform(input_df)
        
        # Make prediction
        prediction = model.predict(X_processed)[0]
        
        # Return the prediction
        return {
            "predicted_price": float(prediction),
            "diamond_features": diamond
        }
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Prediction error: {str(e)}"
        )


if __name__ == "__main__":
    # Run the application with uvicorn when script is executed directly
    uvicorn.run("main:app", host="0.0.0.0", port=3000, reload=True)
```

## Dependency Injection with FastAPI

Welcome back! You've done a fantastic job setting up the data structure and ensuring your model is ready for predictions. Now, let's explore the world of dependency injection in FastAPI, a feature that helps manage resources efficiently and keeps your code clean and organized.

In this exercise, your goal is to implement dependency injection in the predict_price function. This involves using FastAPI's Depends mechanism to inject the model and preprocessor into the function. By doing so, you'll ensure that the model is loaded only when needed, optimizing resource usage and maintaining neat function signatures.

Here's a hint to guide you: examine how the get_model function should be utilized within the predict_price function. Make sure the model and preprocessor are correctly injected and ready to handle incoming requests. Dive in and enhance your API's robustness and efficiency!

```python
"""
FastAPI Application with Model Integration for Diamond Price Prediction

This module extends the basic FastAPI application to include endpoints for
loading the ML model and making diamond price predictions.
"""

from fastapi import FastAPI, HTTPException, Depends
from fastapi.responses import RedirectResponse
from pydantic import BaseModel, Field
import uvicorn
import os
import pandas as pd

from model import load_model_with_metadata

# Initialize FastAPI app
app = FastAPI(
    title="Diamond Price Prediction API",
    description="API for predicting diamond prices based on diamond characteristics",
    version="1.0.0"
)

# Model configuration
MODEL_DIR = "saved_models"
MODEL_NAME = "model"

class DiamondFeatures(BaseModel):
    """
    Pydantic model for diamond features input
    """
    carat: float = Field(..., gt=0, description="Weight of the diamond (0.2-5.0)")
    cut: str = Field(..., description="Quality of the cut (Fair, Good, Very Good, Premium, Ideal)")
    color: str = Field(..., description="Diamond color, from J (worst) to D (best)")
    clarity: str = Field(..., description="Clarity of the diamond (I1, SI2, SI1, VS2, VS1, VVS2, VVS1, IF)")
    depth: float = Field(..., gt=0, description="Total depth percentage")
    table: float = Field(..., gt=0, description="Width of top of diamond relative to widest point")
    x: float = Field(..., gt=0, description="Length in mm")
    y: float = Field(..., gt=0, description="Width in mm")
    z: float = Field(..., gt=0, description="Depth in mm")

class PredictionResponse(BaseModel):
    """
    Pydantic model for prediction response
    """
    predicted_price: float
    diamond_features: DiamondFeatures

def get_model():
    """
    Dependency function to load and return the model and preprocessor on demand
    """
    try:
        os.makedirs(MODEL_DIR, exist_ok=True)
        model_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}.joblib")
        preprocessor_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}_preprocessor.joblib")
        
        # Check if model exists
        if not os.path.exists(model_path) or not os.path.exists(preprocessor_path):
            raise HTTPException(
                status_code=404, 
                detail="Model files not found. Please train and save a model first."
            )
            
        model, preprocessor, _ = load_model_with_metadata(MODEL_DIR, MODEL_NAME)
        return model, preprocessor
    except Exception as e:
        raise HTTPException(status_code=503, detail=f"Error loading model: {str(e)}")

@app.get("/")
async def root():
    """
    Root endpoint, redirects to API documentation
    """
    return RedirectResponse(url="/docs")

@app.get("/health")
async def health_check():
    """
    Health check endpoint to verify the API is running correctly
    """
    model_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}.joblib")
    preprocessor_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}_preprocessor.joblib")
    model_status = "available" if (os.path.exists(model_path) and os.path.exists(preprocessor_path)) else "not available"
    
    return {
        "status": "healthy",
        "api_version": "1.0.0",
        "model_status": model_status
    }

# TODO: Implement dependency injection for the model and preprocessor
# Use the get_model function as a dependency to inject the model and preprocessor
@app.post("/predict", response_model=PredictionResponse)
async def predict_price(diamond: DiamondFeatures):
    """
    Predict the price of a diamond based on its features
    """
    # Load model and preprocessor directly (not using dependency injection)
    try:
        model_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}.joblib")
        preprocessor_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}_preprocessor.joblib")
        
        if not os.path.exists(model_path) or not os.path.exists(preprocessor_path):
            raise HTTPException(
                status_code=404, 
                detail="Model files not found. Please train and save a model first."
            )
        
        model, preprocessor, _ = load_model_with_metadata(MODEL_DIR, MODEL_NAME)
    except Exception as e:
        raise HTTPException(status_code=503, detail=f"Error loading model: {str(e)}")
    
    # Convert input to DataFrame for preprocessing
    input_df = pd.DataFrame([diamond.dict()])
    
    try:
        # Preprocess the input using the same preprocessor from training
        X_processed = preprocessor.transform(input_df)
        
        # Make prediction
        prediction = model.predict(X_processed)[0]
        
        # Return the prediction
        return {
            "predicted_price": float(prediction),
            "diamond_features": diamond
        }
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Prediction error: {str(e)}"
        )


if __name__ == "__main__":
    # Run the application with uvicorn when script is executed directly
    uvicorn.run("main:app", host="0.0.0.0", port=3000, reload=True) 

```

```python
"""
FastAPI Application with Model Integration for Diamond Price Prediction

This module extends the basic FastAPI application to include endpoints for
loading the ML model and making diamond price predictions.
"""

from fastapi import FastAPI, HTTPException, Depends
from fastapi.responses import RedirectResponse
from pydantic import BaseModel, Field
import uvicorn
import os
import pandas as pd

from model import load_model_with_metadata

# Initialize FastAPI app
app = FastAPI(
    title="Diamond Price Prediction API",
    description="API for predicting diamond prices based on diamond characteristics",
    version="1.0.0"
)

# Model configuration
MODEL_DIR = "saved_models"
MODEL_NAME = "model"

class DiamondFeatures(BaseModel):
    """
    Pydantic model for diamond features input
    """
    carat: float = Field(..., gt=0, description="Weight of the diamond (0.2-5.0)")
    cut: str = Field(..., description="Quality of the cut (Fair, Good, Very Good, Premium, Ideal)")
    color: str = Field(..., description="Diamond color, from J (worst) to D (best)")
    clarity: str = Field(..., description="Clarity of the diamond (I1, SI2, SI1, VS2, VS1, VVS2, VVS1, IF)")
    depth: float = Field(..., gt=0, description="Total depth percentage")
    table: float = Field(..., gt=0, description="Width of top of diamond relative to widest point")
    x: float = Field(..., gt=0, description="Length in mm")
    y: float = Field(..., gt=0, description="Width in mm")
    z: float = Field(..., gt=0, description="Depth in mm")

class PredictionResponse(BaseModel):
    """
    Pydantic model for prediction response
    """
    predicted_price: float
    diamond_features: DiamondFeatures

def get_model():
    """
    Dependency function to load and return the model and preprocessor on demand
    """
    try:
        os.makedirs(MODEL_DIR, exist_ok=True)
        model_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}.joblib")
        preprocessor_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}_preprocessor.joblib")
        
        # Check if model exists
        if not os.path.exists(model_path) or not os.path.exists(preprocessor_path):
            raise HTTPException(
                status_code=404, 
                detail="Model files not found. Please train and save a model first."
            )
            
        model, preprocessor, _ = load_model_with_metadata(MODEL_DIR, MODEL_NAME)
        return model, preprocessor
    except Exception as e:
        raise HTTPException(status_code=503, detail=f"Error loading model: {str(e)}")

@app.get("/")
async def root():
    """
    Root endpoint, redirects to API documentation
    """
    return RedirectResponse(url="/docs")

@app.get("/health")
async def health_check():
    """
    Health check endpoint to verify the API is running correctly
    """
    model_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}.joblib")
    preprocessor_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}_preprocessor.joblib")
    model_status = "available" if (os.path.exists(model_path) and os.path.exists(preprocessor_path)) else "not available"
    
    return {
        "status": "healthy",
        "api_version": "1.0.0",
        "model_status": model_status
    }

# TODO: Implement dependency injection for the model and preprocessor
# Use the get_model function as a dependency to inject the model and preprocessor
@app.post("/predict", response_model=PredictionResponse)
async def predict_price(diamond: DiamondFeatures, model_data: tuple = Depends(get_model)):
    """
    Predict the price of a diamond based on its features
    """
    # Use the dependency-injected model and preprocessor
    model, preprocessor = model_data
    
    # Convert input to DataFrame for preprocessing
    input_df = pd.DataFrame([diamond.dict()])
    
    try:
        # Preprocess the input using the same preprocessor from training
        X_processed = preprocessor.transform(input_df)
        
        # Make prediction
        prediction = model.predict(X_processed)[0]
        
        # Return the prediction
        return {
            "predicted_price": float(prediction),
            "diamond_features": diamond
        }
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Prediction error: {str(e)}"
        )


if __name__ == "__main__":
    # Run the application with uvicorn when script is executed directly
    uvicorn.run("main:app", host="0.0.0.0", port=3000, reload=True)
```

## Enhance API Response Structure

Welcome back! You've done a fantastic job so far in building your diamond price prediction API.

Now, in this practice you should define the PredictionResponse class using Pydantic. This class will act as the blueprint for the response your API sends back after making a prediction. It should include fields for the predicted_price and the original diamond_features.

Once you've defined this class, make sure to specify it as the response_model in the predict_price endpoint. This ensures that FastAPI will validate and format the response according to your defined schema.

By completing this, you'll ensure that your API not only makes accurate predictions but also communicates them clearly and consistently. Dive in and make your API's responses as polished as its predictions!

```python
"""
FastAPI Application with Model Integration for Diamond Price Prediction

This module extends the basic FastAPI application to include endpoints for
loading the ML model and making diamond price predictions.
"""

from fastapi import FastAPI, HTTPException, Depends
from fastapi.responses import RedirectResponse
from pydantic import BaseModel, Field
import uvicorn
import os
import pandas as pd

from model import load_model_with_metadata

# Initialize FastAPI app
app = FastAPI(
    title="Diamond Price Prediction API",
    description="API for predicting diamond prices based on diamond characteristics",
    version="1.0.0"
)

# Model configuration
MODEL_DIR = "saved_models"
MODEL_NAME = "model"

class DiamondFeatures(BaseModel):
    """
    Pydantic model for diamond features input
    """
    carat: float = Field(..., gt=0, description="Weight of the diamond (0.2-5.0)")
    cut: str = Field(..., description="Quality of the cut (Fair, Good, Very Good, Premium, Ideal)")
    color: str = Field(..., description="Diamond color, from J (worst) to D (best)")
    clarity: str = Field(..., description="Clarity of the diamond (I1, SI2, SI1, VS2, VS1, VVS2, VVS1, IF)")
    depth: float = Field(..., gt=0, description="Total depth percentage")
    table: float = Field(..., gt=0, description="Width of top of diamond relative to widest point")
    x: float = Field(..., gt=0, description="Length in mm")
    y: float = Field(..., gt=0, description="Width in mm")
    z: float = Field(..., gt=0, description="Depth in mm")

# TODO: Create a PredictionResponse class that inherits from BaseModel
# This class should define the structure of the API response
# It should include fields for the predicted price and the original diamond features


def get_model():
    """
    Dependency function to load and return the model and preprocessor on demand
    """
    try:
        os.makedirs(MODEL_DIR, exist_ok=True)
        model_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}.joblib")
        preprocessor_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}_preprocessor.joblib")
        
        # Check if model exists
        if not os.path.exists(model_path) or not os.path.exists(preprocessor_path):
            raise HTTPException(
                status_code=404, 
                detail="Model files not found. Please train and save a model first."
            )
            
        model, preprocessor, _ = load_model_with_metadata(MODEL_DIR, MODEL_NAME)
        return model, preprocessor
    except Exception as e:
        raise HTTPException(status_code=503, detail=f"Error loading model: {str(e)}")

@app.get("/")
async def root():
    """
    Root endpoint, redirects to API documentation
    """
    return RedirectResponse(url="/docs")

@app.get("/health")
async def health_check():
    """
    Health check endpoint to verify the API is running correctly
    """
    model_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}.joblib")
    preprocessor_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}_preprocessor.joblib")
    model_status = "available" if (os.path.exists(model_path) and os.path.exists(preprocessor_path)) else "not available"
    
    return {
        "status": "healthy",
        "api_version": "1.0.0",
        "model_status": model_status
    }

# TODO: Add the response_model parameter to the endpoint decorator
@app.post("/predict")
async def predict_price(diamond: DiamondFeatures, model_data: tuple = Depends(get_model)):
    """
    Predict the price of a diamond based on its features
    """
    # Use the dependency-injected model and preprocessor
    model, preprocessor = model_data
    
    # Convert input to DataFrame for preprocessing
    input_df = pd.DataFrame([diamond.dict()])
    
    try:
        # Preprocess the input using the same preprocessor from training
        X_processed = preprocessor.transform(input_df)
        
        # Make prediction
        prediction = model.predict(X_processed)[0]
        
        # TODO: Return the prediction according to the response_model
        return float(prediction), diamond
        
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Prediction error: {str(e)}"
        )


if __name__ == "__main__":
    # Run the application with uvicorn when script is executed directly
    uvicorn.run("main:app", host="0.0.0.0", port=3000, reload=True) 
```

```python
"""
FastAPI Application with Model Integration for Diamond Price Prediction

This module extends the basic FastAPI application to include endpoints for
loading the ML model and making diamond price predictions.
"""

from fastapi import FastAPI, HTTPException, Depends
from fastapi.responses import RedirectResponse
from pydantic import BaseModel, Field
import uvicorn
import os
import pandas as pd

from model import load_model_with_metadata

# Initialize FastAPI app
app = FastAPI(
    title="Diamond Price Prediction API",
    description="API for predicting diamond prices based on diamond characteristics",
    version="1.0.0"
)

# Model configuration
MODEL_DIR = "saved_models"
MODEL_NAME = "model"

class DiamondFeatures(BaseModel):
    """
    Pydantic model for diamond features input
    """
    carat: float = Field(..., gt=0, description="Weight of the diamond (0.2-5.0)")
    cut: str = Field(..., description="Quality of the cut (Fair, Good, Very Good, Premium, Ideal)")
    color: str = Field(..., description="Diamond color, from J (worst) to D (best)")
    clarity: str = Field(..., description="Clarity of the diamond (I1, SI2, SI1, VS2, VS1, VVS2, VVS1, IF)")
    depth: float = Field(..., gt=0, description="Total depth percentage")
    table: float = Field(..., gt=0, description="Width of top of diamond relative to widest point")
    x: float = Field(..., gt=0, description="Length in mm")
    y: float = Field(..., gt=0, description="Width in mm")
    z: float = Field(..., gt=0, description="Depth in mm")

# TODO: Create a PredictionResponse class that inherits from BaseModel
# This class should define the structure of the API response
# It should include fields for the predicted price and the original diamond features
class PredictionResponse(BaseModel):
    """
    Pydantic model for prediction response
    """
    predicted_price: float
    diamond_features: DiamondFeatures


def get_model():
    """
    Dependency function to load and return the model and preprocessor on demand
    """
    try:
        os.makedirs(MODEL_DIR, exist_ok=True)
        model_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}.joblib")
        preprocessor_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}_preprocessor.joblib")
        
        # Check if model exists
        if not os.path.exists(model_path) or not os.path.exists(preprocessor_path):
            raise HTTPException(
                status_code=404, 
                detail="Model files not found. Please train and save a model first."
            )
            
        model, preprocessor, _ = load_model_with_metadata(MODEL_DIR, MODEL_NAME)
        return model, preprocessor
    except Exception as e:
        raise HTTPException(status_code=503, detail=f"Error loading model: {str(e)}")

@app.get("/")
async def root():
    """
    Root endpoint, redirects to API documentation
    """
    return RedirectResponse(url="/docs")

@app.get("/health")
async def health_check():
    """
    Health check endpoint to verify the API is running correctly
    """
    model_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}.joblib")
    preprocessor_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}_preprocessor.joblib")
    model_status = "available" if (os.path.exists(model_path) and os.path.exists(preprocessor_path)) else "not available"
    
    return {
        "status": "healthy",
        "api_version": "1.0.0",
        "model_status": model_status
    }

# TODO: Add the response_model parameter to the endpoint decorator
@app.post("/predict", response_model=PredictionResponse)
async def predict_price(diamond: DiamondFeatures, model_data: tuple = Depends(get_model)):
    """
    Predict the price of a diamond based on its features
    """
    # Use the dependency-injected model and preprocessor
    model, preprocessor = model_data
    
    # Convert input to DataFrame for preprocessing
    input_df = pd.DataFrame([diamond.dict()])
    
    try:
        # Preprocess the input using the same preprocessor from training
        X_processed = preprocessor.transform(input_df)
        
        # Make prediction
        prediction = model.predict(X_processed)[0]
        
        # TODO: Return the prediction according to the response_model
        return {
            "predicted_price": float(prediction),
            "diamond_features": diamond
        }
        
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Prediction error: {str(e)}"
        )


if __name__ == "__main__":
    # Run the application with uvicorn when script is executed directly
    uvicorn.run("main:app", host="0.0.0.0", port=3000, reload=True)
```

## Create a Diamond Price Predictor

You've done an excellent job so far in this unit, especially with setting up the FastAPI application and understanding the basics of model integration. Now, it's time to bring everything together and create a fully functional predict_price endpoint in your FastAPI application. This is your opportunity to apply what you've learned and build a reliable prediction service.

Here's what you need to accomplish:

Define a response model using Pydantic to structure the output. This should include both the predicted_price and the original diamond_features.

Implement dependency injection to efficiently load the machine learning model and preprocessor using the get_model function.

Write the prediction logic that processes the input data, applies the preprocessor, and generates a prediction using the model.

Ensure robust error handling to gracefully manage any issues that arise during prediction, providing clear feedback to users.

This is your chance to showcase your skills and create a seamless prediction endpoint that integrates all the concepts you've learned. Dive in and make your API shine!

```python
"""
FastAPI Application with Model Integration for Diamond Price Prediction

This module extends the basic FastAPI application to include endpoints for
loading the ML model and making diamond price predictions.
"""

import os
import pandas as pd
import uvicorn
from fastapi import FastAPI, HTTPException, Depends
from fastapi.responses import RedirectResponse
from pydantic import BaseModel, Field

from model import load_model_with_metadata

# Initialize FastAPI app
app = FastAPI(
    title="Diamond Price Prediction API",
    description="API for predicting diamond prices based on diamond characteristics",
    version="1.0.0"
)

# Model configuration
MODEL_DIR = "saved_models"
MODEL_NAME = "model"

class DiamondFeatures(BaseModel):
    """
    Pydantic model for diamond features input.
    """
    carat: float = Field(..., gt=0, description="Weight of the diamond (0.2-5.0)")
    cut: str = Field(..., description="Quality of the cut (Fair, Good, Very Good, Premium, Ideal)")
    color: str = Field(..., description="Diamond color, from J (worst) to D (best)")
    clarity: str = Field(..., description="Clarity of the diamond (I1, SI2, SI1, VS2, VS1, VVS2, VVS1, IF)")
    depth: float = Field(..., gt=0, description="Total depth percentage")
    table: float = Field(..., gt=0, description="Width of top of diamond relative to widest point")
    x: float = Field(..., gt=0, description="Length in mm")
    y: float = Field(..., gt=0, description="Width in mm")
    z: float = Field(..., gt=0, description="Depth in mm")

# TODO: Create a PredictionResponse Pydantic model to structure the API response
# It should include fields for the predicted price and the original diamond features


def get_model():
    """
    Dependency function to load and return the model and preprocessor on demand.
    """
    try:
        os.makedirs(MODEL_DIR, exist_ok=True)
        model_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}.joblib")
        preprocessor_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}_preprocessor.joblib")
        
        # Ensure that the required files exist
        if not os.path.exists(model_path) or not os.path.exists(preprocessor_path):
            raise HTTPException(
                status_code=404, 
                detail="Model files not found. Please train and save a model first."
            )
            
        model, preprocessor, _ = load_model_with_metadata(MODEL_DIR, MODEL_NAME)
        return model, preprocessor
    except Exception as e:
        raise HTTPException(status_code=503, detail=f"Error loading model: {str(e)}")

@app.get("/")
async def root():
    """
    Root endpoint, redirects to API documentation.
    """
    return RedirectResponse(url="/docs")

@app.get("/health")
async def health_check():
    """
    Health check endpoint to verify the API is running correctly.
    """
    model_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}.joblib")
    preprocessor_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}_preprocessor.joblib")
    model_status = "available" if (os.path.exists(model_path) and os.path.exists(preprocessor_path)) else "not available"
    
    return {
        "status": "healthy",
        "api_version": "1.0.0",
        "model_status": model_status
    }

# TODO: Implement the predict_price endpoint with the following features:
# 1. Use the appropriate HTTP method and path
# 2. Specify the response model for validation and documentation
# 3. Use dependency injection to load the model and preprocessor
# 4. Process the input data and make predictions
# 5. Include proper error handling
# 6. Return the prediction with the original diamond features


if __name__ == "__main__":
    uvicorn.run("main:app", host="0.0.0.0", port=3000, reload=True)

```

```python
"""
FastAPI Application with Model Integration for Diamond Price Prediction

This module extends the basic FastAPI application to include endpoints for
loading the ML model and making diamond price predictions.
"""

import os
import pandas as pd
import uvicorn
from fastapi import FastAPI, HTTPException, Depends
from fastapi.responses import RedirectResponse
from pydantic import BaseModel, Field

from model import load_model_with_metadata

# Initialize FastAPI app
app = FastAPI(
    title="Diamond Price Prediction API",
    description="API for predicting diamond prices based on diamond characteristics",
    version="1.0.0"
)

# Model configuration
MODEL_DIR = "saved_models"
MODEL_NAME = "model"

class DiamondFeatures(BaseModel):
    """
    Pydantic model for diamond features input.
    """
    carat: float = Field(..., gt=0, description="Weight of the diamond (0.2-5.0)")
    cut: str = Field(..., description="Quality of the cut (Fair, Good, Very Good, Premium, Ideal)")
    color: str = Field(..., description="Diamond color, from J (worst) to D (best)")
    clarity: str = Field(..., description="Clarity of the diamond (I1, SI2, SI1, VS2, VS1, VVS2, VVS1, IF)")
    depth: float = Field(..., gt=0, description="Total depth percentage")
    table: float = Field(..., gt=0, description="Width of top of diamond relative to widest point")
    x: float = Field(..., gt=0, description="Length in mm")
    y: float = Field(..., gt=0, description="Width in mm")
    z: float = Field(..., gt=0, description="Depth in mm")

# TODO: Create a PredictionResponse Pydantic model to structure the API response
# It should include fields for the predicted price and the original diamond features
class PredictionResponse(BaseModel):
    """
    Pydantic model for prediction response
    """
    predicted_price: float
    diamond_features: DiamondFeatures


def get_model():
    """
    Dependency function to load and return the model and preprocessor on demand.
    """
    try:
        os.makedirs(MODEL_DIR, exist_ok=True)
        model_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}.joblib")
        preprocessor_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}_preprocessor.joblib")
        
        # Ensure that the required files exist
        if not os.path.exists(model_path) or not os.path.exists(preprocessor_path):
            raise HTTPException(
                status_code=404, 
                detail="Model files not found. Please train and save a model first."
            )
            
        model, preprocessor, _ = load_model_with_metadata(MODEL_DIR, MODEL_NAME)
        return model, preprocessor
    except Exception as e:
        raise HTTPException(status_code=503, detail=f"Error loading model: {str(e)}")

@app.get("/")
async def root():
    """
    Root endpoint, redirects to API documentation.
    """
    return RedirectResponse(url="/docs")

@app.get("/health")
async def health_check():
    """
    Health check endpoint to verify the API is running correctly.
    """
    model_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}.joblib")
    preprocessor_path = os.path.join(MODEL_DIR, f"{MODEL_NAME}_preprocessor.joblib")
    model_status = "available" if (os.path.exists(model_path) and os.path.exists(preprocessor_path)) else "not available"
    
    return {
        "status": "healthy",
        "api_version": "1.0.0",
        "model_status": model_status
    }

# TODO: Implement the predict_price endpoint with the following features:
# 1. Use the appropriate HTTP method and path
# 2. Specify the response model for validation and documentation
# 3. Use dependency injection to load the model and preprocessor
# 4. Process the input data and make predictions
# 5. Include proper error handling
# 6. Return the prediction with the original diamond features
@app.post("/predict", response_model=PredictionResponse)
async def predict_price(diamond: DiamondFeatures, model_data: tuple = Depends(get_model)):
    """
    Predict the price of a diamond based on its features.
    """
    # Use the dependency-injected model and preprocessor
    model, preprocessor = model_data
    
    # Convert input to DataFrame for preprocessing
    input_df = pd.DataFrame([diamond.dict()])
    
    try:
        # Preprocess the input using the same preprocessor from training
        X_processed = preprocessor.transform(input_df)
        
        # Make prediction
        prediction = model.predict(X_processed)[0]
        
        # Return the prediction according to the response_model
        return {
            "predicted_price": float(prediction),
            "diamond_features": diamond
        }
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Prediction error: {str(e)}"
        )


if __name__ == "__main__":
    uvicorn.run("main:app", host="0.0.0.0", port=3000, reload=True)
```